In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from tensorflow.keras.applications import ResNet50,MobileNet, DenseNet201, InceptionV3, NASNetLarge, InceptionResNetV2, NASNetMobile
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
from functools import partial
from sklearn import metrics
from collections import Counter
import json
import itertools
import h5py


%matplotlib inline

In [ ]:
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".png" or ".tif":
            img = read(PATH)
           
            img = cv2.resize(img, (RESIZE,RESIZE))
           
            IMG.append(np.array(img))
    return IMG

In [ ]:
cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis'
print(cases_path)
cases = os.listdir(cases_path)
print(cases)
for j in range(len(cases)):
  magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis'+'/'+cases[j]
  print(magnification_path)
  magnifications = os.listdir(magnification_path)
  print(magnifications)
  for k in range(len(magnifications)): 
    path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis'+'/'+cases[j]+'/'+magnifications[k]
    print(path)
    if k==0:
      adenosis = np.array(Dataset_loader(path, 128))
    else:
      x = np.array(Dataset_loader(path, 128))
      adenosis = np.concatenate((adenosis,x),axis=0)
    print(len(adenosis))
  if j==0:
    adenosis_all = adenosis
  else:
    adenosis_all = np.concatenate((adenosis_all,adenosis),axis=0)
  print(len(adenosis_all))

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis
['SOB_B_A_14-22549CD', 'SOB_B_A_14-29960CD', 'SOB_B_A_14-22549AB', 'SOB_B_A_14-22549G']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549CD
['40X', '400X', '100X', '200X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549CD/40X


100%|██████████| 35/35 [00:15<00:00,  2.32it/s]


35
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549CD/400X


100%|██████████| 30/30 [00:10<00:00,  2.74it/s]


65
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549CD/100X


...

64
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549G/200X


100%|██████████| 32/32 [00:14<00:00,  2.22it/s]


96
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/adenosis/SOB_B_A_14-22549G/100X


100%|██████████| 34/34 [00:15<00:00,  2.20it/s]

130
444


In [ ]:
cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma'
print(cases_path)
cases = os.listdir(cases_path)
print(cases)
for j in range(len(cases)):
  magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma'+'/'+cases[j]
  print(magnification_path)
  magnifications = os.listdir(magnification_path)
  print(magnifications)
  for k in range(len(magnifications)): 
    path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma'+'/'+cases[j]+'/'+magnifications[k]
    print(path)
    if k==0:
      fibroadenoma = np.array(Dataset_loader(path, 128))
    else:
      x = np.array(Dataset_loader(path, 128))
      fibroadenoma = np.concatenate((fibroadenoma,x),axis=0)
    print(len(fibroadenoma))
  if j==0:
    fibroadenoma_all = fibroadenoma
  else:
    fibroadenoma_all = np.concatenate((fibroadenoma_all,fibroadenoma),axis=0)
  print(len(fibroadenoma_all))

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma
['SOB_B_F_14-25197', 'SOB_B_F_14-23222AB', 'SOB_B_F_14-29960AB', 'SOB_B_F_14-9133', 'SOB_B_F_14-23060AB', 'SOB_B_F_14-23060CD', 'SOB_B_F_14-21998CD', 'SOB_B_F_14-21998EF', 'SOB_B_F_14-14134E', 'SOB_B_F_14-14134']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197
['400X', '40X', '200X', '100X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197/400X


100%|██████████| 33/33 [00:14<00:00,  2.26it/s]


33
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197/40X


100%|██████████| 22/22 [00:09<00:00,  2.24it/s]


55
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-25197/200X


...

73
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-14134/100X


100%|██████████| 31/31 [00:08<00:00,  3.69it/s]


104
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/fibroadenoma/SOB_B_F_14-14134/200X


100%|██████████| 38/38 [00:13<00:00,  2.91it/s]

142
1014


In [ ]:
cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor'
print(cases_path)
cases = os.listdir(cases_path)
print(cases)
for j in range(len(cases)):
  magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor'+'/'+cases[j]
  print(magnification_path)
  magnifications = os.listdir(magnification_path)
  print(magnifications)
  for k in range(len(magnifications)): 
    path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor'+'/'+cases[j]+'/'+magnifications[k]
    print(path)
    if k==0:
      phyllodes_tumor = np.array(Dataset_loader(path, 128))
    else:
      x = np.array(Dataset_loader(path, 128))
      phyllodes_tumor = np.concatenate((phyllodes_tumor,x),axis=0)
    print(len(phyllodes_tumor))
  if j==0:
    phyllodes_tumor_all = phyllodes_tumor
  else:
    phyllodes_tumor_all = np.concatenate((phyllodes_tumor_all,phyllodes_tumor),axis=0)
  print(len(phyllodes_tumor_all))

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor
['SOB_B_PT_14-29315EF', 'SOB_B_PT_14-21998AB', 'SOB_B_PT_14-22704']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-29315EF
['40X', '400X', '100X', '200X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-29315EF/40X


100%|██████████| 13/13 [00:05<00:00,  2.47it/s]


13
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-29315EF/400X


100%|██████████| 17/17 [00:07<00:00,  2.26it/s]


30
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-29315EF/100X


...

77
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-22704/200X


100%|██████████| 42/42 [00:12<00:00,  3.45it/s]


119
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/phyllodes_tumor/SOB_B_PT_14-22704/100X


100%|██████████| 39/39 [00:10<00:00,  3.87it/s]

158
453


In [ ]:
cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma'
print(cases_path)
cases = os.listdir(cases_path)
print(cases)
for j in range(len(cases)):
  magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma'+'/'+cases[j]
  print(magnification_path)
  magnifications = os.listdir(magnification_path)
  print(magnifications)
  for k in range(len(magnifications)): 
    path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma'+'/'+cases[j]+'/'+magnifications[k]
    print(path)
    if k==0:
      tubular_adenoma = np.array(Dataset_loader(path, 128))
    else:
      x = np.array(Dataset_loader(path, 128))
      tubular_adenoma = np.concatenate((tubular_adenoma,x),axis=0)
    print(len(tubular_adenoma))
  if j==0:
    tubular_adenoma_all = tubular_adenoma
  else:
    tubular_adenoma_all = np.concatenate((tubular_adenoma_all,tubular_adenoma),axis=0)
  print(len(tubular_adenoma_all))

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma
['SOB_B_TA_14-19854C', 'SOB_B_TA_14-21978AB', 'SOB_B_TA_14-16184CD', 'SOB_B_TA_14-3411F', 'SOB_B_TA_14-16184', 'SOB_B_TA_14-15275', 'SOB_B_TA_14-13200']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma/SOB_B_TA_14-19854C
['40X', '400X', '100X', '200X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma/SOB_B_TA_14-19854C/40X


100%|██████████| 16/16 [00:07<00:00,  2.22it/s]


16
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma/SOB_B_TA_14-19854C/400X


100%|██████████| 16/16 [00:07<00:00,  2.20it/s]


32
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma/SOB_B_TA_14-19854C/100X


...

35
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma/SOB_B_TA_14-13200/40X


100%|██████████| 15/15 [00:07<00:00,  2.11it/s]


50
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/benign/SOB/tubular_adenoma/SOB_B_TA_14-13200/400X


100%|██████████| 15/15 [00:05<00:00,  2.53it/s]

65
569


In [ ]:
print(adenosis_all.shape,
tubular_adenoma_all.shape,
fibroadenoma_all.shape,
phyllodes_tumor_all.shape)

(444, 128, 128, 3) (569, 128, 128, 3) (1014, 128, 128, 3) (453, 128, 128, 3)


In [ ]:
cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma'
print(cases_path)
cases = os.listdir(cases_path)
print(cases)
for j in range(len(cases)):
  magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma'+'/'+cases[j]
  print(magnification_path)
  magnifications = os.listdir(magnification_path)
  print(magnifications)
  for k in range(len(magnifications)): 
    path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma'+'/'+cases[j]+'/'+magnifications[k]
    print(path)
    if k==0:
      papillary_carcinoma = np.array(Dataset_loader(path, 128))
    else:
      x = np.array(Dataset_loader(path, 128))
      papillary_carcinoma = np.concatenate((papillary_carcinoma,x),axis=0)
    print(len(papillary_carcinoma))
  if j==0:
    papillary_carcinoma_all = papillary_carcinoma
  else:
    papillary_carcinoma_all = np.concatenate((papillary_carcinoma_all,papillary_carcinoma),axis=0)
  print(len(papillary_carcinoma_all))

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma
['SOB_M_PC_15-190EF', 'SOB_M_PC_14-9146', 'SOB_M_PC_14-15704', 'SOB_M_PC_14-12465', 'SOB_M_PC_14-19440', 'SOB_M_PC_14-15687B']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF
['40X', '400X', '100X', '200X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF/40X


100%|██████████| 19/19 [00:00<00:00, 79.10it/s]


19
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF/400X


100%|██████████| 15/15 [00:00<00:00, 78.40it/s]


34
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_15-190EF/100X


...

33
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_14-15687B/200X


100%|██████████| 14/14 [00:06<00:00,  2.24it/s]


47
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/papillary_carcinoma/SOB_M_PC_14-15687B/400X


100%|██████████| 15/15 [00:06<00:00,  2.31it/s]

62
560


In [ ]:
cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma'
print(cases_path)
cases = os.listdir(cases_path)
print(cases)
for j in range(len(cases)):
  magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma'+'/'+cases[j]
  print(magnification_path)
  magnifications = os.listdir(magnification_path)
  print(magnifications)
  for k in range(len(magnifications)): 
    path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma'+'/'+cases[j]+'/'+magnifications[k]
    print(path)
    if k==0:
      ductal_carcinoma = np.array(Dataset_loader(path, 128))
    else:
      x = np.array(Dataset_loader(path, 128))
      ductal_carcinoma = np.concatenate((ductal_carcinoma,x),axis=0)
    print(len(ductal_carcinoma))
  if j==0:
    ductal_carcinoma_all = ductal_carcinoma
  else:
    ductal_carcinoma_all = np.concatenate((ductal_carcinoma_all,ductal_carcinoma),axis=0)
  print(len(ductal_carcinoma_all))

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma
['SOB_M_DC_14-8168', 'SOB_M_DC_14-9461', 'SOB_M_DC_14-5695', 'SOB_M_DC_14-6241', 'SOB_M_DC_14-5287', 'SOB_M_DC_14-5694', 'SOB_M_DC_14-4364', 'SOB_M_DC_14-2773', 'SOB_M_DC_14-4372', 'SOB_M_DC_14-3909', 'SOB_M_DC_14-2985', 'SOB_M_DC_14-2980', 'SOB_M_DC_14-2523', 'SOB_M_DC_14-20636', 'SOB_M_DC_14-18650', 'SOB_M_DC_14-20629', 'SOB_M_DC_14-17915', 'SOB_M_DC_14-17901', 'SOB_M_DC_14-16875', 'SOB_M_DC_14-16601', 'SOB_M_DC_14-16716', 'SOB_M_DC_14-17614', 'SOB_M_DC_14-16448', 'SOB_M_DC_14-16336', 'SOB_M_DC_14-15792', 'SOB_M_DC_14-16188', 'SOB_M_DC_14-14946', 'SOB_M_DC_14-13993', 'SOB_M_DC_14-15572', 'SOB_M_DC_14-14926', 'SOB_M_DC_14-15696', 'SOB_M_DC_14-14015', 'SOB_M_DC_14-12312', 'SOB_M_DC_14-13412', 'SOB_M_DC_14-11951', 'SOB_M_DC_14-11520', 'SOB_M_DC_14-10926', 'SOB_M_DC_14-11031']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-8168

100%|██████████| 9/9 [00:04<00:00,  2.20it/s]


9
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-8168/40X


100%|██████████| 10/10 [00:04<00:00,  2.23it/s]


19
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-8168/100X


...

29
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-11031/100X


100%|██████████| 17/17 [00:07<00:00,  2.27it/s]


46
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/ductal_carcinoma/SOB_M_DC_14-11031/200X


100%|██████████| 14/14 [00:05<00:00,  2.68it/s]

60
3451


In [ ]:
cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma'
print(cases_path)
cases = os.listdir(cases_path)
print(cases)
for j in range(len(cases)):
  magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma'+'/'+cases[j]
  print(magnification_path)
  magnifications = os.listdir(magnification_path)
  print(magnifications)
  for k in range(len(magnifications)): 
    path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma'+'/'+cases[j]+'/'+magnifications[k]
    print(path)
    if k==0:
      mucinous_carcinoma = np.array(Dataset_loader(path, 128))
    else:
      x = np.array(Dataset_loader(path, 128))
      mucinous_carcinoma = np.concatenate((mucinous_carcinoma,x),axis=0)
    print(len(mucinous_carcinoma))
  if j==0:
    mucinous_carcinoma_all = mucinous_carcinoma
  else:
    mucinous_carcinoma_all = np.concatenate((mucinous_carcinoma_all,mucinous_carcinoma),axis=0)
  print(len(mucinous_carcinoma_all))

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma
['SOB_M_MC_14-19979C', 'SOB_M_MC_14-19979', 'SOB_M_MC_14-18842D', 'SOB_M_MC_14-18842', 'SOB_M_MC_14-16456', 'SOB_M_MC_14-13418DE', 'SOB_M_MC_14-13413', 'SOB_M_MC_14-12773', 'SOB_M_MC_14-10147']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma/SOB_M_MC_14-19979C
['400X', '40X', '200X', '100X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma/SOB_M_MC_14-19979C/400X


100%|██████████| 15/15 [00:06<00:00,  2.39it/s]


15
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma/SOB_M_MC_14-19979C/40X


100%|██████████| 17/17 [00:07<00:00,  2.30it/s]


32
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma/SOB_M_MC_14-19979C/200X


...

27
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma/SOB_M_MC_14-10147/400X


100%|██████████| 15/15 [00:07<00:00,  1.93it/s]


42
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/mucinous_carcinoma/SOB_M_MC_14-10147/200X


100%|██████████| 22/22 [00:08<00:00,  2.50it/s]

64
792


In [ ]:
cases_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma'
print(cases_path)
cases = os.listdir(cases_path)
print(cases)
for j in range(len(cases)):
  magnification_path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma'+'/'+cases[j]
  print(magnification_path)
  magnifications = os.listdir(magnification_path)
  print(magnifications)
  for k in range(len(magnifications)): 
    path = '/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma'+'/'+cases[j]+'/'+magnifications[k]
    print(path)
    if k==0:
      lobular_carcinoma = np.array(Dataset_loader(path, 128))
    else:
      x = np.array(Dataset_loader(path, 128))
      lobular_carcinoma = np.concatenate((lobular_carcinoma,x),axis=0)
    print(len(lobular_carcinoma))
  if j==0:
    lobular_carcinoma_all = lobular_carcinoma
  else:
    lobular_carcinoma_all = np.concatenate((lobular_carcinoma_all,lobular_carcinoma),axis=0)
  print(len(lobular_carcinoma_all))

/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma
['SOB_M_LC_14-16196', 'SOB_M_LC_14-15570C', 'SOB_M_LC_14-12204', 'SOB_M_LC_14-15570', 'SOB_M_LC_14-13412']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-16196
['40X', '400X', '100X', '200X']
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-16196/40X


100%|██████████| 22/22 [00:09<00:00,  2.42it/s]


22
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-16196/400X


100%|██████████| 17/17 [00:07<00:00,  2.38it/s]


39
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-16196/100X


...

58
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-13412/200X


100%|██████████| 32/32 [00:10<00:00,  3.16it/s]


90
/content/drive/MyDrive/Datasets/BreaKHis_v1/histology_slides/breast/malignant/SOB/lobular_carcinoma/SOB_M_LC_14-13412/100X


100%|██████████| 33/33 [00:08<00:00,  3.84it/s]

123
626


In [ ]:
print(papillary_carcinoma_all.shape, ductal_carcinoma_all.shape, lobular_carcinoma_all.shape, mucinous_carcinoma_all.shape)

(560, 128, 128, 3) (3451, 128, 128, 3) (626, 128, 128, 3) (792, 128, 128, 3)


In [ ]:
f1 = h5py.File("/content/drive/MyDrive/Types_BreakHis.hdf5", "w")

In [ ]:
 dset1 = f1.create_dataset("adenosis", adenosis_all.shape , dtype='i', data=adenosis_all)
 dset2 = f1.create_dataset("fibroadenoma", fibroadenoma_all.shape, dtype='i', data = fibroadenoma_all)
 dset3 = f1.create_dataset("tubular_adenoma", tubular_adenoma_all.shape , dtype='i', data= tubular_adenoma_all)
 dset4 = f1.create_dataset("phyllodes_tumor", phyllodes_tumor_all.shape, dtype='i', data = phyllodes_tumor_all)
 dset5 = f1.create_dataset("lobular_carcinoma", lobular_carcinoma_all.shape , dtype='i', data= lobular_carcinoma_all)
 dset6 = f1.create_dataset("ductal_carcinoma", ductal_carcinoma_all.shape, dtype='i', data = ductal_carcinoma_all)
 dset7 = f1.create_dataset("papillary_carcinoma", papillary_carcinoma_all.shape , dtype='i', data= papillary_carcinoma_all)
 dset8 = f1.create_dataset("mucinous_carcinoma", mucinous_carcinoma_all.shape, dtype='i', data = mucinous_carcinoma_all)

In [ ]:
list(f1.keys())

['adenosis',
 'ductal_carcinoma',
 'fibroadenoma',
 'lobular_carcinoma',
 'mucinous_carcinoma',
 'papillary_carcinoma',
 'phyllodes_tumor',
 'tubular_adenoma']

In [ ]:
f1.close()

In [ ]:
f2 = h5py.File("/content/drive/MyDrive/Types_BreakHis.hdf5", "r")

In [ ]:
list(f2.keys())

['adenosis',
 'ductal_carcinoma',
 'fibroadenoma',
 'lobular_carcinoma',
 'mucinous_carcinoma',
 'papillary_carcinoma',
 'phyllodes_tumor',
 'tubular_adenoma']

In [ ]:
f2.close()